In [18]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, MetaData

In [1]:
from pathlib import Path
Path('my_data.sqlite').touch()

In [43]:
import sqlite3
conn = sqlite3.connect('my_data.sqlite')
c = conn.cursor()

In [3]:
# new cell block
# different from one below because i adds a id_card col as primary key
# this allows the db file to work with sqlalchemy 
c.execute('''CREATE TABLE data_science (
    work_year INTEGER
    experience_level TEXT,
    employment_type TEXT,
    job_title TEXT,
    salary INTEGER,
    salary_currency TEXT,
    salary_in_usd INTEGER,
    employee_residence TEXT,
    remote_ratio INTEGER,
    company_location TEXT,
    company_size TEXT,
    id_card Serial,
    CONSTRAINT data_science_pkey PRIMARY KEY (id_card)
);
''')

In [4]:
# c.execute('''CREATE TABLE data_science (
#     work_year INTEGER
#     experience_level TEXT,
#     employment_type TEXT,
#     job_title TEXT,
#     salary INTEGER,
#     salary_currency TEXT,
#     salary_in_usd INTEGER,
#     employee_residence TEXT,
#     remote_ratio INTEGER,
#     company_location TEXT,
#     company_size TEXT
# );
# ''')

In [4]:
import pandas as pd
# load the data into a Pandas DataFrame
data = pd.read_csv(r"Data\ds_salaries.csv")
# write the data to a sqlite table
data.to_sql('data_salaries', conn, if_exists='append', index = False)

3755

In [44]:
data = c.execute('''SELECT work_year, salary FROM data_salaries WHERE work_year=2022''').fetchall()
data

[(2022, 1650000),
 (2022, 300000),
 (2022, 54000),
 (2022, 84000),
 (2022, 275000),
 (2022, 57000),
 (2022, 70000),
 (2022, 164000),
 (2022, 130000),
 (2022, 34320),
 (2022, 48000),
 (2022, 106000),
 (2022, 192000),
 (2022, 168000),
 (2022, 85000),
 (2022, 375000),
 (2022, 180000),
 (2022, 23000),
 (2022, 155000),
 (2022, 80000),
 (2022, 48000),
 (2022, 6000),
 (2022, 1250000),
 (2022, 110000),
 (2022, 24000),
 (2022, 1440000),
 (2022, 100000),
 (2022, 133000),
 (2022, 1125000),
 (2022, 130000),
 (2022, 1100000),
 (2022, 125000),
 (2022, 175000),
 (2022, 155000),
 (2022, 153600),
 (2022, 106800),
 (2022, 150000),
 (2022, 140000),
 (2022, 122000),
 (2022, 94500),
 (2022, 170000),
 (2022, 130000),
 (2022, 145000),
 (2022, 128000),
 (2022, 145000),
 (2022, 100000),
 (2022, 175000),
 (2022, 120000),
 (2022, 145000),
 (2022, 115000),
 (2022, 150000),
 (2022, 150000),
 (2022, 100000),
 (2022, 150000),
 (2022, 127500),
 (2022, 126500),
 (2022, 51000),
 (2022, 260000),
 (2022, 175000),
 (2022,

### Getting country location data

In [29]:
import requests
import json
import pandas as pd

In [30]:
# grab country_data from our api to merge with coordinate df
url = "http://127.0.0.1:5000/country_data"
r = requests.get(url).json()

In [31]:
df = pd.DataFrame(r).drop(columns = ['remote_ratio','salary_in_usd'])
df

,company_location
0,AE
1,AL
2,AM
3,AR
4,AS
...,...
67,TH
68,TR
69,UA
70,US


In [32]:
#read in csv from https://gist.github.com/tadast/8827699
coord_df = pd.read_csv(r"C:\Users\Stan\Downloads\countries_codes_and_coordinates.csv")
coord_df.head()

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,"""AF""","""AFG""","""4""","""33""","""65"""
1,Albania,"""AL""","""ALB""","""8""","""41""","""20"""
2,Algeria,"""DZ""","""DZA""","""12""","""28""","""3"""
3,American Samoa,"""AS""","""ASM""","""16""","""-14.3333""","""-170"""
4,Andorra,"""AD""","""AND""","""20""","""42.5""","""1.6"""


In [33]:
coord_df = coord_df[['Alpha-2 code','Latitude (average)', 'Longitude (average)']]

In [34]:
coord_df["Alpha-2 code"] = coord_df["Alpha-2 code"].str.replace('"', '')
coord_df["Latitude (average)"] = coord_df["Latitude (average)"].str.replace('"', '')
coord_df["Longitude (average)"] = coord_df["Longitude (average)"].str.replace('"', '')
coord_df["Alpha-2 code"] = coord_df["Alpha-2 code"].str.replace(' ', '')
coord_df["Latitude (average)"] = coord_df["Latitude (average)"].str.replace(' ', '')
coord_df["Longitude (average)"] = coord_df["Longitude (average)"].str.replace(' ', '')

In [35]:
coord_df = coord_df.rename(columns = {"Alpha-2 code":"company_location"})
coord_df

,company_location,Latitude (average),Longitude (average)
0,AF,33,65
1,AL,41,20
2,DZ,28,3
3,AS,-14.3333,-170
4,AD,42.5,1.6
...,...,...,...
251,WF,-13.3,-176.2
252,EH,24.5,-13
253,YE,15,48
254,ZM,-15,30


In [36]:
merged_df = pd.merge(df, coord_df, on = "company_location", how = 'inner')
merged_df

,company_location,Latitude (average),Longitude (average)
0,AE,24,54
1,AL,41,20
2,AM,40,45
3,AR,-34,-64
4,AS,-14.3333,-170
...,...,...,...
70,TR,39,35
71,UA,49,32
72,US,38,-97
73,VN,16,106


In [37]:
merged_df[merged_df.duplicated(subset = ['company_location', 'Latitude (average)', 'Longitude (average)'])]

,company_location,Latitude (average),Longitude (average)
10,BO,-17,-65
64,RU,60,100
74,VN,16,106


In [38]:
df_cleaned = merged_df.drop([merged_df.index[10], merged_df.index[64], merged_df.index[74]])
df_cleaned

,company_location,Latitude (average),Longitude (average)
0,AE,24,54
1,AL,41,20
2,AM,40,45
3,AR,-34,-64
4,AS,-14.3333,-170
...,...,...,...
69,TH,15,100
70,TR,39,35
71,UA,49,32
72,US,38,-97


In [39]:
df_cleaned = df_cleaned.astype({'Latitude (average)':'float64','Longitude (average)':'float64'})

In [42]:
df_cleaned = df_cleaned.rename(columns = {'company_location':'Country','Latitude (average)':'Latitude','Longitude (average)':'Longitude'})
df_cleaned.head()

,Country,Latitude,Longitude
0,AE,24.0000,54.0
1,AL,41.0000,20.0
2,AM,40.0000,45.0
3,AR,-34.0000,-64.0
4,AS,-14.3333,-170.0


In [51]:
df_cleaned.to_csv("Data/country_locations.csv")

### Load new csv into database

In [46]:
# new cell block
# different from one below because i adds a id_card col as primary key
# this allows the db file to work with sqlalchemy 
c.execute('''CREATE TABLE country_locations (
    Country TEXT,
    Latitude DECIMAL,
    Longitude DECIMAL
    );
''')

In [52]:
# load the data into a Pandas DataFrame
data = pd.read_csv(r"Data\country_locations.csv").drop(columns = "Unnamed: 0")
# write the data to a sqlite table
data.to_sql('country_locations', conn, if_exists='append', index = False)

72

In [53]:
data = c.execute('''SELECT * FROM country_locations''').fetchall()
data

[('AE', 24, 54),
 ('AL', 41, 20),
 ('AM', 40, 45),
 ('AR', -34, -64),
 ('AS', -14.3333, -170),
 ('AT', 47.3333, 13.3333),
 ('AU', -27, 133),
 ('BA', 44, 18),
 ('BE', 50.8333, 4),
 ('BO', -17, -65),
 ('BR', -10, -55),
 ('BS', 24.25, -76),
 ('CA', 60, -95),
 ('CF', 7, 21),
 ('CH', 47, 8),
 ('CL', -30, -71),
 ('CN', 35, 105),
 ('CO', 4, -72),
 ('CR', 10, -84),
 ('CZ', 49.75, 15.5),
 ('DE', 51, 9),
 ('DK', 56, 10),
 ('DZ', 28, 3),
 ('EE', 59, 26),
 ('EG', 27, 30),
 ('ES', 40, -4),
 ('FI', 64, 26),
 ('FR', 46, 2),
 ('GB', 54, -2),
 ('GH', 8, -2),
 ('GR', 39, 22),
 ('HK', 22.25, 114.1667),
 ('HN', 15, -86.5),
 ('HR', 45.1667, 15.5),
 ('HU', 47, 20),
 ('ID', -5, 120),
 ('IE', 53, -8),
 ('IL', 31.5, 34.75),
 ('IN', 20, 77),
 ('IQ', 33, 44),
 ('IR', 32, 53),
 ('IT', 42.8333, 12.8333),
 ('JP', 36, 138),
 ('KE', 1, 38),
 ('LT', 56, 24),
 ('LU', 49.75, 6.1667),
 ('LV', 57, 25),
 ('MA', 32, -5),
 ('MD', 47, 29),
 ('MK', 41.8333, 22),
 ('MT', 35.8333, 14.5833),
 ('MX', 23, -102),
 ('MY', 2.5, 112.5)

In [55]:
conn.close()